# EDA

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

***Load datasets and explore***

In [17]:
loans_test = pd.read_csv("./data/raw_data/loans_test.csv")
loans_train = pd.read_csv("./data/raw_data/loans_train.csv")
loans_valid = pd.read_csv("./data/raw_data/loans_valid.csv")


In [18]:
# 使用原始列值创建唯一ID，避免index重复问题
loans_train["LoanID"] = loans_train["index"]
loans_valid["LoanID"] = loans_valid["index"]  
loans_test["LoanID"] = loans_test["Id"]  # 测试集使用Id列


***Drop columns with 100% missing data***

In [19]:
# Drop columns with 100% missing data directly
columns_to_drop = ['ReliefRefinanceIndicator', 'PreHARP_Flag']

# Drop columns directly from original datasets
loans_train.drop(columns=columns_to_drop, inplace=True)
loans_valid.drop(columns=columns_to_drop, inplace=True)
loans_test.drop(columns=columns_to_drop, inplace=True)

## Cleaning Data

### Static Columns

In [20]:
import pandas as pd
import numpy as np
import re

# ============================================
# STATIC (ORIGINATION) VARIABLES CLEANING
# ============================================

DATE_COLS = ["FirstPaymentDate", "MaturityDate"]
NUM_SPECIAL_9999 = ["CreditScore"]
NUM_SPECIAL_999 = ["MI_Pct", "OriginalCLTV", "OriginalDTI", "OriginalLTV"]
NUM_POSITIVE_ONLY = ["OriginalUPB"]
NUM_CLIP_BOUNDS = {
    "OriginalDTI": (0, 65),
    "OriginalLoanTerm": (1, 480),
    "NumberOfUnits": (1, 4),
    "NumberOfBorrowers": (1, 10),
}
CAT_NA_CODES = {
    "FirstTimeHomebuyerFlag": {"9"},
    "OccupancyStatus": {"9"},
    "Channel": {"9"},
    "PropertyType": {"99"},
    "LoanPurpose": {"9"},
}
OHE_WHITELIST = {
    # "FirstTimeHomebuyerFlag": ["N", "Y"],
    # "OccupancyStatus": ["I", "P", "S"],
    # "Channel": ["B", "C", "R"],
    # "PPM_Flag": ["N"],
    # "ProductType": ["FRM"],
    # "PropertyType": ["CO", "CP", "MH", "PU", "SF"],
    # "LoanPurpose": ["C", "N", "P"],
    # "SuperConformingFlag": ["Y"],
    # "ProgramIndicator": ["9", "F", "H"],
    # "InterestOnlyFlag": ["N"],
    # "BalloonIndicator": ["7", "N", "Y"],
}


def _safe_to_datetime_yyyymm(s: pd.Series):
    """Convert YYYYMM strings to datetime; invalid values become NaT."""
    return pd.to_datetime(s.astype("string"), format="%Y%m", errors="coerce")


def _apply_special_missing(df: pd.DataFrame):
    """Replace coded missing values (9999, 999, 99, 9) with NaN."""
    for c in NUM_SPECIAL_9999:
        if c in df:
            df[c] = df[c].replace(9999, np.nan)
    for c in NUM_SPECIAL_999:
        if c in df:
            df[c] = df[c].replace(999, np.nan)
    for c, bad_set in CAT_NA_CODES.items():
        if c in df:
            df[c] = df[c].where(~df[c].astype("string").isin(bad_set), np.nan)
    return df


def _clip_and_validate(df: pd.DataFrame):
    """Clip numeric columns to valid ranges and enforce positivity."""
    for c in NUM_POSITIVE_ONLY:
        if c in df:
            df[c] = pd.to_numeric(df[c], errors="coerce")
            df.loc[df[c] <= 0, c] = np.nan
    for c, (lo, hi) in NUM_CLIP_BOUNDS.items():
        if c in df:
            df[c] = pd.to_numeric(df[c], errors="coerce")
            df[c] = df[c].clip(lower=lo, upper=hi)
    return df


def _parse_dates(df: pd.DataFrame):
    """Convert YYYYMM columns to pandas datetime."""
    for c in DATE_COLS:
        if c in df:
            df[c] = _safe_to_datetime_yyyymm(df[c])
    return df


def _ohe_whitelist(df: pd.DataFrame, drop_original: bool = False):
    """Strict one-hot encoding based on whitelist mappings."""
    for col, cats in OHE_WHITELIST.items():
        if col not in df:
            for v in cats:
                df[f"{col}_{v}"] = 0
            continue
        col_values = df[col].astype("string")
        for v in cats:
            df[f"{col}_{v}"] = (col_values == str(v)).fillna(False).astype(int)
        if drop_original:
            df.drop(columns=[col], inplace=True, errors="ignore")
    return df


def _add_optional_derivatives(df: pd.DataFrame):
    """Add derived features like year, month, and designed loan term."""
    if "FirstPaymentDate" in df:
        df["FirstPaymentYear"] = df["FirstPaymentDate"].dt.year
        df["FirstPaymentMonth"] = df["FirstPaymentDate"].dt.month
    if "MaturityDate" in df:
        df["MaturityYear"] = df["MaturityDate"].dt.year
        df["MaturityMonth"] = df["MaturityDate"].dt.month
    if "FirstPaymentDate" in df and "MaturityDate" in df:
        term_days = (df["MaturityDate"] - df["FirstPaymentDate"]).dt.days
        df["DesignedLoanTermMonths"] = (term_days / 30.4375).round().astype("float")
    return df


# ------------------------------------------------------------
# Main cleaning entry function
# ------------------------------------------------------------

def clean_static_features(
    df: pd.DataFrame,
    *,
    add_derivatives: bool = True,
    drop_original_categoricals_after_ohe: bool = False,
) -> pd.DataFrame:  # 修复：只返回一个DataFrame
    """
    Clean static (origination) features only.
    Automatically removes panel columns before cleaning.

    Returns
    -------
    cleaned_static_df : pd.DataFrame
        Cleaned static-only DataFrame.
    """
    df = df.copy()

    # --- Step 0: Remove panel columns ---
    panel_cols = [c for c in df.columns if re.match(r"^\d+_", c)]
    if len(panel_cols) > 0:
        print(f"⚙️ Detected {len(panel_cols)} panel columns — removed before static cleaning.")
        df = df.drop(columns=panel_cols)

    # --- Step 1–5: Normal static cleaning workflow ---
    df = _apply_special_missing(df)
    df = _parse_dates(df)
    df = _clip_and_validate(df)
    df = _ohe_whitelist(df, drop_original=drop_original_categoricals_after_ohe)
    if add_derivatives:
        df = _add_optional_derivatives(df)

    return df



In [21]:
# 修复：正确的数据处理流程 - 分离静态特征和面板数据
print("=== 步骤1: 分离和清洗静态特征 ===")

# 清洗静态特征（会自动移除面板列）
loans_train_static_clean = clean_static_features(
    loans_train,
    add_derivatives=True,                         # 是否生成派生特征（年/月/期限）
    drop_original_categoricals_after_ohe=False    # 是否删除原始类别列
)

loans_valid_static_clean = clean_static_features(
    loans_valid,
    add_derivatives=True,                         # 是否生成派生特征（年/月/期限）
    drop_original_categoricals_after_ohe=False    # 是否删除原始类别列
)

loans_test_static_clean = clean_static_features(
    loans_test,
    add_derivatives=True,                         # 是否生成派生特征（年/月/期限）
    drop_original_categoricals_after_ohe=False    # 是否删除原始类别列
)

=== 步骤1: 分离和清洗静态特征 ===
⚙️ Detected 112 panel columns — removed before static cleaning.
⚙️ Detected 112 panel columns — removed before static cleaning.
⚙️ Detected 112 panel columns — removed before static cleaning.


# Panel Columns

| 字段名                                | 含义                       | 清洗方式                     | 派生建议                            |
| ---------------------------------- | ------------------------ | ------------------------ | ------------------------------- |
| `N_MonthlyReportingPeriod`         | 报告期 (YYYYMM)             | 转换为日期 (`pd.to_datetime`) | 可派生年份、月份                        |
| `N_CurrentActualUPB`               | 当前未偿本金余额                 | 若 ≤0 → 视为结清；保持为 float    | 可派生 `is_fully_paid`, `upb_diff` |
| `N_CurrentInterestRate`            | 当前利率                     | clip 到合理范围 (0, 20)       | 可分析利率变化趋势                       |
| `N_CurrentNonInterestBearingUPB`   | 非计息本金                    | 保留，可派生 `is_modified`     |                                 |
| `N_EstimatedLTV`                   | 当前估算 LTV (1–998, 999=未知) | 替换 999→NaN               | 可计算 LTV 变化                      |
| `N_InterestBearingUPB`             | 计息部分本金                   | 保留                       | 可用于计算占比                         |
| `N_LoanAge`                        | 自首付以来的月龄                 | 保留                       | 与时间排序一致                         |
| `N_RemainingMonthsToLegalMaturity` | 距法定到期剩余月数                | clip 到 (0, 480)          | 可用于计算剩余期限比例                     |


***Extract Panel Data***

In [22]:
import re
import pandas as pd

def extract_panel_data(df: pd.DataFrame, id_col: str = "LoanID") -> pd.DataFrame:
    """
    从宽表中提取 LoanID + 面板列（0_*, 1_*, …, 13_*）
    """
    df = df.copy()
    df.columns = df.columns.str.strip()

    # 确保 LoanID 存在
    if id_col not in df.columns:
        if "index" in df.columns:
            df = df.rename(columns={"index": id_col})
        else:
            df = df.reset_index().rename(columns={"index": id_col})

    # 正则匹配所有面板期列（0_~13_）
    panel_cols = [c for c in df.columns if re.match(r"^(1[0-3]|[0-9])_", c)]
    keep_cols = [id_col] + panel_cols

    panel_df = df[keep_cols].copy()
    print(f"✅ 提取出面板数据：{len(panel_cols)} 个面板特征列，{panel_df.shape[0]} 行。")
    return panel_df


In [23]:
loans_train_panel = extract_panel_data(loans_train)
loans_valid_panel = extract_panel_data(loans_valid)
loans_test_panel = extract_panel_data(loans_test)

✅ 提取出面板数据：112 个面板特征列，30504 行。
✅ 提取出面板数据：112 个面板特征列，5370 行。
✅ 提取出面板数据：112 个面板特征列，13426 行。


***data clean***

In [24]:
import pandas as pd
import numpy as np
import re


def clean_performance_panel_wide(df: pd.DataFrame) -> pd.DataFrame:
    """
    清理宽表面板数据：
    1. 替换非法值（999, 负数 → NaN）
    2. 解析每期的 MonthlyReportingPeriod 列
    """

    df = df.copy()

    # 遍历所有 0–13 期列
    for col in df.columns:
        # 只处理形如 "n_" 开头的列
        if re.match(r"^\d+_", col):
            # 数值列：替换非法值
            df[col] = pd.to_numeric(df[col], errors="ignore")
            if np.issubdtype(df[col].dtype, np.number):
                df.loc[df[col] == 999, col] = np.nan
                df.loc[df[col] < 0, col] = np.nan

            # 如果是月份列（例如 0_MonthlyReportingPeriod）
            if col.endswith("MonthlyReportingPeriod"):
                try:
                    df[col] = pd.to_datetime(
                        df[col].astype("Int64").astype(str),
                        format="%Y%m", errors="coerce"
                    )
                except Exception:
                    pass  # 避免非标准格式报错

    return df


In [25]:
print("=== 步骤3: 清洗面板数据 ===")

# 应用面板数据清洗函数
loans_train_panel_clean = clean_performance_panel_wide(loans_train_panel)
loans_valid_panel_clean = clean_performance_panel_wide(loans_valid_panel)
loans_test_panel_clean = clean_performance_panel_wide(loans_test_panel)

print(f"面板数据清洗完成:")
print(f"训练集面板数据: {loans_train_panel_clean.shape}")
print(f"验证集面板数据: {loans_valid_panel_clean.shape}")
print(f"测试集面板数据: {loans_test_panel_clean.shape}")


=== 步骤3: 清洗面板数据 ===


C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_n

面板数据清洗完成:
训练集面板数据: (30504, 113)
验证集面板数据: (5370, 113)
测试集面板数据: (13426, 113)


C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_n

In [26]:
loans_train = clean_performance_panel_wide(loans_train)
loans_valid = clean_performance_panel_wide(loans_valid)
loans_test= clean_performance_panel_wide(loans_test)

C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\DELL\AppData\Local\Temp\ipykernel_103220\1388767309.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_n

## Save Data

In [27]:
loans_train.to_csv("./data/cleaned_data/loans_train.csv")
loans_valid.to_csv("./data/cleaned_data/loans_valid.csv")
loans_test.to_csv("./data/cleaned_data/loans_test.csv")

In [28]:

# 合并面板数据和静态特征
train = loans_train_panel_clean.merge(
    loans_train_static_clean,
    on="LoanID",
    how="left"
)

valid = loans_valid_panel_clean.merge(
    loans_valid_static_clean,
    on="LoanID",
    how="left"
)

test = loans_test_panel_clean.merge(
    loans_test_static_clean,
    on="LoanID",
    how="left"
)

# 验证合并结果
print(f"合并前检查:")
print(f"  面板数据: train={loans_train_panel_clean.shape}, valid={loans_valid_panel_clean.shape}, test={loans_test_panel_clean.shape}")
print(f"  静态数据: train={loans_train_static_clean.shape}, valid={loans_valid_static_clean.shape}, test={loans_test_static_clean.shape}")

print(f"\n合并后检查:")
print(f"  最终数据: train={train.shape}, valid={valid.shape}, test={test.shape}")

print(f"\nLoanID匹配验证:")
print(f"  训练集: {train['LoanID'].nunique()} / {len(train)} (应该相等)")
print(f"  验证集: {valid['LoanID'].nunique()} / {len(valid)} (应该相等)")
print(f"  测试集: {test['LoanID'].nunique()} / {len(test)} (应该相等)")

# 检查是否有数据丢失
print(f"\n数据完整性检查:")
print(f"  训练集合并成功率: {len(train)} / {len(loans_train)} = {len(train)/len(loans_train)*100:.1f}%")
print(f"  验证集合并成功率: {len(valid)} / {len(loans_valid)} = {len(valid)/len(loans_valid)*100:.1f}%")
print(f"  测试集合并成功率: {len(test)} / {len(loans_test)} = {len(test)/len(loans_test)*100:.1f}%")


合并前检查:
  面板数据: train=(30504, 113), valid=(5370, 113), test=(13426, 113)
  静态数据: train=(30504, 37), valid=(5370, 37), test=(13426, 36)

合并后检查:
  最终数据: train=(30504, 149), valid=(5370, 149), test=(13426, 148)

LoanID匹配验证:
  训练集: 30504 / 30504 (应该相等)
  验证集: 5370 / 5370 (应该相等)
  测试集: 13426 / 13426 (应该相等)

数据完整性检查:
  训练集合并成功率: 30504 / 30504 = 100.0%
  验证集合并成功率: 5370 / 5370 = 100.0%
  测试集合并成功率: 13426 / 13426 = 100.0%


In [29]:
print("=== 步骤5: 保存清洗后的数据 ===")

# 保存最终清洗后的数据
train.to_csv("./data/cleaned_data/loans_train.csv", index=False)
valid.to_csv("./data/cleaned_data/loans_valid.csv", index=False)
test.to_csv("./data/cleaned_data/loans_test.csv", index=False)

print("✅ 数据保存完成!")
print(f"  训练集: ./data/cleaned_data/loans_train.csv ({train.shape})")
print(f"  验证集: ./data/cleaned_data/loans_valid.csv ({valid.shape})")
print(f"  测试集: ./data/cleaned_data/loans_test.csv ({test.shape})")

# 最终数据质量检查
print(f"\n=== 最终数据质量报告 ===")
print(f"特征数量: {train.shape[1]} (包含LoanID, index, target)")
print(f"静态特征数量: {len([col for col in train.columns if not col.startswith(('0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_'))])}")
print(f"面板特征数量: {len([col for col in train.columns if col.startswith(('0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_'))])}")
print(f"缺失值总数: train={train.isnull().sum().sum()}, valid={valid.isnull().sum().sum()}, test={test.isnull().sum().sum()}")

# 显示最终列结构
print(f"\n=== 最终列结构示例 ===")
static_cols = [col for col in train.columns if not col.startswith(('0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_'))]
panel_cols = [col for col in train.columns if col.startswith(('0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_'))]

print(f"静态特征列 ({len(static_cols)}个):")
for i, col in enumerate(static_cols[:10]):  # 只显示前10个
    print(f"  {i+1:2d}. {col}")
if len(static_cols) > 10:
    print(f"  ... 还有 {len(static_cols)-10} 个静态特征")

print(f"\n面板特征列 ({len(panel_cols)}个):")
print(f"  时间点0: {[col for col in panel_cols if col.startswith('0_')]}")
print(f"  时间点1: {[col for col in panel_cols if col.startswith('1_')]}")
print(f"  ... 共14个时间点，每期8个特征")


=== 步骤5: 保存清洗后的数据 ===
✅ 数据保存完成!
  训练集: ./data/cleaned_data/loans_train.csv ((30504, 149))
  验证集: ./data/cleaned_data/loans_valid.csv ((5370, 149))
  测试集: ./data/cleaned_data/loans_test.csv ((13426, 148))

=== 最终数据质量报告 ===
特征数量: 149 (包含LoanID, index, target)
静态特征数量: 37
面板特征数量: 112
缺失值总数: train=69418, valid=12359, test=30330

=== 最终列结构示例 ===
静态特征列 (37个):
   1. LoanID
   2. index
   3. target
   4. CreditScore
   5. FirstPaymentDate
   6. FirstTimeHomebuyerFlag
   7. MaturityDate
   8. MSA
   9. MI_Pct
  10. NumberOfUnits
  ... 还有 27 个静态特征

面板特征列 (112个):
  时间点0: ['0_CurrentActualUPB', '0_CurrentInterestRate', '0_CurrentNonInterestBearingUPB', '0_EstimatedLTV', '0_InterestBearingUPB', '0_LoanAge', '0_MonthlyReportingPeriod', '0_RemainingMonthsToLegalMaturity']
  时间点1: ['1_CurrentActualUPB', '1_CurrentInterestRate', '1_CurrentNonInterestBearingUPB', '1_EstimatedLTV', '1_InterestBearingUPB', '1_LoanAge', '1_MonthlyReportingPeriod', '1_RemainingMonthsToLegalMaturity']
  ... 共14个时间点，每期8个特征
